In [32]:
import random
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

def generate_y(data_in, batch_size = 10, len_seq = 5):
    data_out = []
    for d in data_in:
        a = 0
        for i in d:
            a = (a + i) % 2
            data_out.append(a)
    return np.array(data_out).reshape(-1, batch_size, len_seq)

len_train = 1000
batch_size = 10
len_seq = 5
data = [random.randint(0, 1) for i in range(len_train)]
x = np.array(data).reshape(-1, batch_size, len_seq)
y = generate_y(x)

data = [random.randint(0, 1) for i in range(20)]
x_test = np.array(data).reshape(-1, len_seq)



In [47]:


# Define paramaters for the model
learning_rate = 0.01
n_epochs = 51
hidden_size = 4


def fullLayer(input_data, output_size, act, std = 0.1):
    ##defining the full linear Layer here
    w = tf.Variable(tf.random_normal([input_data.get_shape().as_list()[1], output_size], stddev = std))
    b = tf.Variable(tf.zeros([output_size]))
    return act(tf.matmul(input_data, w) + b)


#define placeholder
# all image are 28 * 28 so x has 784 dimension
X = tf.placeholder(tf.float32, [None, len_seq], name='X_placeholder')
Y = tf.placeholder(tf.float32, [None, len_seq], name='Y_placeholder')
stddev = tf.Variable(0.1, name = "stddev")

## the layers
h = fullLayer(X, hidden_size, tf.nn.sigmoid)
logits = fullLayer(h, len_seq, tf.nn.sigmoid)

loss = tf.reduce_sum((logits - Y) * (logits - Y))

##defining optimizer
## using gradient descent with learning rate of 0.5 to minimize loss
gradient = tf.train.GradientDescentOptimizer(learning_rate)
optimizer = gradient.minimize(loss)

#just some config for not getting whole server
gpu_options = tf.GPUOptions(allow_growth=True)
config = tf.ConfigProto(gpu_options=gpu_options)

with tf.Session(config = config) as sess:
    ##training the model 

    ##starting time
    start_time = time.time()
    ##initialize the variables
    sess.run(tf.global_variables_initializer())

    for i in range(n_epochs):  # train the model n_epochs times
        total_loss = 0
        total_acc = 0
        for j in range(len(x)):
            ##training batches
            _, loss_batch, pred = sess.run([optimizer, loss, logits], feed_dict={X: x[j], Y: y[j]})
            total_loss += loss_batch
            
        if i % 5 == 0:
            print('epoch {}, Average loss : {}'.format(i, total_loss / len(x)))

    print('Total time: {0} seconds'.format(time.time() - start_time))
    print('Optimization Finished!')

    # test the model
    ##number of test batches
    x_test = [[1,0,1,1,1]]
    print(x_test)
    p = sess.run([logits], feed_dict={X: x_test})
    p = p[0] > 1/2
    
    s = []
    for i in range(len(p)):
        for j in range(len(p[0, :])):
            s.append(int(p[i,j]))
    print(np.array(s).reshape(p.shape))
            
    
            
sess.close()


epoch 0, Average loss : 12.564545726776123
epoch 5, Average loss : 12.465834045410157
epoch 10, Average loss : 12.461934232711792
epoch 15, Average loss : 12.459651708602905
epoch 20, Average loss : 12.45728359222412
epoch 25, Average loss : 12.454673290252686
epoch 30, Average loss : 12.451744937896729
epoch 35, Average loss : 12.44842562675476
epoch 40, Average loss : 12.444636821746826
epoch 45, Average loss : 12.44029598236084
epoch 50, Average loss : 12.435315370559692
Total time: 1.6319503784179688 seconds
Optimization Finished!
[[1, 0, 1, 1, 1]]
[[0 1 0 0 1]]
